# Background subtraction

This notebook implements the method developed in script Background.ipynb in a loop to handle a batch of images.

It expects .ARW images as input, and generates corresponding .bkg_subtracted.fits files.

In [2]:
import os, glob
import time, datetime
import math

import multiprocessing as mp
from multiprocessing import Pool

import numpy as np

from scipy import stats

from astropy.io import fits
from astropy.table import Table
from astropy.stats import SigmaClip
from astropy.convolution import Gaussian2DKernel, interpolate_replace_nans

from photutils.background import Background2D, MedianBackground, ModeEstimatorBackground

import rawpy
import exifread

from worker_background import Worker

from settings import data_dirpath, ARW_suffix, bkgsub_suffix

## Initialization

In [3]:
# these were determined in script White_light_images as the normalization
# factors that render the smoothest background.
red_norm =  1.34
blue_norm = 1.25

# lets try with best for star @ 4300K
# red_norm =  1.9
# blue_norm = 1.9

# parameters to control background subtraction
bkg_cell_footprint = (35, 35)
bkg_filter = (9, 9)

bkg_sigma_clip = SigmaClip(sigma=5.)
bkg_kernel = Gaussian2DKernel(x_stddev=2)
bkg_estimator = ModeEstimatorBackground()

In [4]:
# ARW images to subtract background from
image_list = list(glob.glob(data_dirpath + '/*' + ARW_suffix))
image_list.sort()

image_list

['/Users/busko/Projects/astrophotography_data/Andromeda_2022/106MSDCF/DSC00150.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/106MSDCF/DSC00151.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/106MSDCF/DSC00152.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/106MSDCF/DSC00153.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/106MSDCF/DSC00154.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/106MSDCF/DSC00155.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/106MSDCF/DSC00156.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/106MSDCF/DSC00157.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/106MSDCF/DSC00158.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/106MSDCF/DSC00159.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/106MSDCF/DSC00160.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/106M

In [5]:
# read base image - we need this in order to access the camera color matrix
raw = rawpy.imread(image_list[int(len(image_list)/2)]) # mid-point
imarray_base = raw.raw_image_visible.astype(float)

In [6]:
# masks that isolate the RGB pixels
colors_array = raw.raw_colors_visible

red_mask = np.where(colors_array == 0, 1, 0)

green_mask_1 = np.where(colors_array == 1, 1, 0)
green_mask_2 = np.where(colors_array == 3, 1, 0)
green_mask = green_mask_1 | green_mask_2

blue_mask = np.where(colors_array == 2, 1, 0)

## Functions

All functions are defined in module 'worker_background.py'. The multiprocessing framework requires that everything that runs on a separate process has to have its own address space. 

## Main loop

In [ ]:
# number of processors
nproc = 8    # (Mac M1)

# split file list into nproc chunks; one for each processor
image_list_chuncks = np.array_split(image_list, nproc)

results = []
pool = Pool(nproc)

for p in range(nproc):
    worker = Worker(image_list_chuncks[p], ARW_suffix, bkgsub_suffix, 
                    red_mask, green_mask, blue_mask, 
                    red_norm, blue_norm,
                    bkg_cell_footprint, bkg_filter, bkg_sigma_clip, bkg_kernel, bkg_estimator)

    r = pool.apply_async(worker)
    results.append(r)

for r in results:
    r.wait()

pool.close()    